In [1]:
#import xgboost as xgb
import xarray as xr
import pandas as pd
import numpy as np
import gc
# from flaml import AutoML
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import matplotlib.pyplot as plt
import logging 
import pickle
# logging.disable(logging.CRITICAL)
#import shap
from tqdm import tqdm

In [2]:
model_path = ""
data_path_prefix = "" 
p_dict = {"a":model_path+"all_gridcell/", 
          "c":model_path+"clusters/",
          "r":model_path+"regions/"}

def get_model_path(spatial_scale,time_scale,gas_flag,loc):
    if spatial_scale == "r":
        p = p_dict[spatial_scale]+time_scale+"_aod_emission_met"+gas_flag+"_"+loc+".pkl"
    else:
        p = p_dict[spatial_scale]+time_scale+"_aod_emission_met"+gas_flag+".pkl"
    return p
        
def load_model(spatial_scale,time_scale,gas_flag,loc):
    if spatial_scale == "r":
        p = p_dict[spatial_scale]+time_scale+"_aod_emission_met"+gas_flag+"_"+loc+".pkl"
    else:
        p = p_dict[spatial_scale]+time_scale+"_aod_emission_met"+gas_flag+".pkl"

    with open(p, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        automl = pickle.load(f)
        model_name =  automl._best_estimator
    return automl, model_name

def get_feature_list(time_scale,gas_flag):
    aod_ls = ['AOT_C', 'AOT_DUST_C']
    met_ls = ['T2M', 'PBLH', 'U10M', 'V10M', 'PRECTOT', 'RH']
    gas_ls = ['CO_trop', 'SO2_trop', 'NO2_trop', 'CH2O_trop', 'NH3_trop']

    # select based on time scale
    if time_scale == "monthly":
        emission = ['EmisDST_Natural', 
                    'EmisNO_Fert', 'EmisNO_Lightning', 'EmisNO_Ship', 'EmisNO_Soil',
                    'EmisBC_Anthro', 'EmisBC_BioBurn', 
                    'EmisCH2O_Anthro', 'EmisCH2O_BioBurn', 
                    'EmisCO_Anthro', 'EmisCO_BioBurn', 'EmisCO_Ship', 
                    'EmisNH3_Anthro', 'EmisNH3_BioBurn', 'EmisNH3_Natural', 
                    'EmisNO_Aircraft', 'EmisNO_Anthro', 'EmisNO_BioBurn', 
                    'EmisOC_Anthro', 'EmisOC_BioBurn',  
                    'EmisSO2_Aircraft', 'EmisSO2_Anthro', 'EmisSO2_BioBurn',
                    'EmisSO4_Anthro']
        
         # select based on gas or not
        if gas_flag=="_gas":
            return emission+met_ls+gas_ls
        else:
            return emission+met_ls

    else:
        emission = ['EmisDST_Natural', 
                    'EmisNO_Fert', 'EmisNO_Lightning', 'EmisNO_Ship', 'EmisNO_Soil']
        if gas_flag=="_gas":
            return emission+met_ls+gas_ls
        else:
            return emission+met_ls

def load_data(spatial_scale,time_scale,gas_flag,loc,data_type):
    # get feature list and include label
    feature_ls = get_feature_list(time_scale,gas_flag)
    # select based on spatial_scale
    if spatial_scale =="r":
        if time_scale=="monthly_le":
            data_path = data_path_prefix+"c_r_monthly_"+data_type+".gzip"
        else:
            data_path = data_path_prefix+"c_r_"+time_scale+"_"+data_type+".gzip"
        df = pd.read_parquet(data_path)[feature_ls+["PM25","region"]]
        return df[df["region"]==loc], feature_ls
    elif spatial_scale == "c":
        if time_scale=="monthly_le":
            data_path = data_path_prefix+"c_r_monthly_"+data_type+".gzip"
        else:
            data_path = data_path_prefix+"c_r_"+time_scale+"_"+data_type+".gzip"
        df = pd.read_parquet(data_path)[feature_ls+["PM25"]]
        return df, feature_ls
    else:
        if time_scale=="monthly_le":
            data_path = data_path_prefix+"monthly_"+data_type+".gzip"
        else:
            data_path = data_path_prefix+time_scale+"_"+data_type+".gzip"
        df = pd.read_parquet(data_path)[feature_ls+["PM25"]]
        return df, feature_ls
    
def model_performance(df, model, feature_ls, spatial_scale):
    y_true = df["PM25"]
    y_pred = model.predict(df[feature_ls])
    print(spatial_scale, "r2_score:",
          r2_score(y_true, y_pred))
    print(spatial_scale, "root mean_squared_error:",
          mean_squared_error(y_true, y_pred, squared = False))
    print(spatial_scale, "mean_absolute_error:",
          mean_absolute_error(y_true, y_pred))

In [3]:
SpatialScale_ls = []
TimeScale_ls = []
Gas_ls = []
R2_ls = []
RMSE_ls = []
MAE_ls = []
AOD_ls = []
df_ori = pd.read_csv("./data/LR.csv")
df_ori["AOD"] = "Y"

## regional model performance

In [4]:
for loc in ["E","S","W","N"]:
    print("============================")
    print("start location:",loc,"\n")
    for time_scale in ["daily","monthly_le","monthly"]:
        print("===",time_scale,"===")
        print("#########")
        for gas_flag in ["_gas"]:
            print("##gas_flag:",gas_flag)
            for spatial_scale in ["r"]:
                print("**spatial scale:", spatial_scale)
                df_train, feature_ls = load_data(spatial_scale,time_scale,gas_flag,loc,"train")
                df_test, feature_ls = load_data(spatial_scale,time_scale,gas_flag,loc,"test")
                X_train = df_train[feature_ls]
                y_train = df_train["PM25"]
                X_test = df_test[feature_ls]
                y_true = df_test["PM25"]

                # train the model
                scaler = StandardScaler()
                scaler.fit(X_train)
                X_train_new = scaler.transform(X_train)
                X_test_new = scaler.transform(X_test)
                X2 = sm.add_constant(X_train_new)
                reg = sm.OLS(y_train, X2).fit()

                X_test2 = sm.add_constant(X_test_new)
                y_pred = reg.predict(X_test2)
                print(spatial_scale, "r2_score:",
                      round(r2_score(y_true, y_pred),2))
                print(spatial_scale, "root mean_squared_error:",
                      round(mean_squared_error(y_true, y_pred, squared = False),2))
                print(spatial_scale, "mean_absolute_error:",
                      round(mean_absolute_error(y_true, y_pred),2))
                
                SpatialScale_ls.append(loc)
                TimeScale_ls.append(time_scale)
                Gas_ls.append("Y")
                R2_ls.append(round(r2_score(y_true, y_pred),2))
                RMSE_ls.append(round(mean_squared_error(y_true, y_pred, squared = False),2))
                MAE_ls.append(round(mean_absolute_error(y_true, y_pred),2))
                AOD_ls.append("N")
                
                del df_train, df_test, scaler, reg, X_train_new, X_test_new, X_train, y_train, X_test, y_true
                gc.collect()
        print("============================")

start location: E 

=== daily ===
#########
##gas_flag: _gas
**spatial scale: r
r r2_score: 0.39
r root mean_squared_error: 7.44
r mean_absolute_error: 5.54
=== monthly_le ===
#########
##gas_flag: _gas
**spatial scale: r
r r2_score: 0.33
r root mean_squared_error: 6.23
r mean_absolute_error: 4.6
=== monthly ===
#########
##gas_flag: _gas
**spatial scale: r
r r2_score: 0.31
r root mean_squared_error: 6.34
r mean_absolute_error: 4.25
start location: S 

=== daily ===
#########
##gas_flag: _gas
**spatial scale: r
r r2_score: 0.33
r root mean_squared_error: 5.72
r mean_absolute_error: 4.44
=== monthly_le ===
#########
##gas_flag: _gas
**spatial scale: r
r r2_score: 0.35
r root mean_squared_error: 3.74
r mean_absolute_error: 3.17
=== monthly ===
#########
##gas_flag: _gas
**spatial scale: r
r r2_score: 0.37
r root mean_squared_error: 3.69
r mean_absolute_error: 3.14
start location: W 

=== daily ===
#########
##gas_flag: _gas
**spatial scale: r
r r2_score: 0.29
r root mean_squared_error: 8

## all gridcells, and clusters

In [5]:
for spatial_scale in ["a","c"]:
    print("============================")
    print("**spatial scale:", spatial_scale)
    for time_scale in ["daily","monthly_le","monthly"]:
        print("===",time_scale,"===")
        print("#########")
        for gas_flag in ["_gas"]:
            print("##gas_flag:",gas_flag)
            df_train, feature_ls = load_data(spatial_scale,time_scale,gas_flag,"None","train")
            df_test, feature_ls = load_data(spatial_scale,time_scale,gas_flag,"None","test")
            X_train = df_train[feature_ls]
            y_train = df_train["PM25"]
            X_test = df_test[feature_ls]
            y_true = df_test["PM25"]

            # train the model
            scaler = StandardScaler()
            scaler.fit(X_train)
            X_train_new = scaler.transform(X_train)
            X_test_new = scaler.transform(X_test)
            X2 = sm.add_constant(X_train_new)
            reg = sm.OLS(y_train, X2).fit()

            X_test2 = sm.add_constant(X_test_new)
            y_pred = reg.predict(X_test2)
            print(spatial_scale, "r2_score:",
                  round(r2_score(y_true, y_pred),2))
            print(spatial_scale, "root mean_squared_error:",
                  round(mean_squared_error(y_true, y_pred, squared = False),2))
            print(spatial_scale, "mean_absolute_error:",
                  round(mean_absolute_error(y_true, y_pred),2))

            SpatialScale_ls.append(spatial_scale)
            TimeScale_ls.append(time_scale)
            Gas_ls.append("Y")
            R2_ls.append(round(r2_score(y_true, y_pred),2))
            RMSE_ls.append(round(mean_squared_error(y_true, y_pred, squared = False),2))
            MAE_ls.append(round(mean_absolute_error(y_true, y_pred),2))
            AOD_ls.append("N")
            
            del df_train, df_test, scaler, reg, X_train_new, X_test_new, X_train, y_train, X_test, y_true
            gc.collect()
    print("============================")

**spatial scale: a
=== daily ===
#########
##gas_flag: _gas
a r2_score: 0.66
a root mean_squared_error: 8.33
a mean_absolute_error: 6.06
=== monthly_le ===
#########
##gas_flag: _gas
a r2_score: 0.77
a root mean_squared_error: 6.07
a mean_absolute_error: 4.67
=== monthly ===
#########
##gas_flag: _gas
a r2_score: 0.78
a root mean_squared_error: 5.88
a mean_absolute_error: 4.46
**spatial scale: c
=== daily ===
#########
##gas_flag: _gas
c r2_score: 0.44
c root mean_squared_error: 9.33
c mean_absolute_error: 7.01
=== monthly_le ===
#########
##gas_flag: _gas
c r2_score: 0.58
c root mean_squared_error: 6.39
c mean_absolute_error: 4.95
=== monthly ===
#########
##gas_flag: _gas
c r2_score: 0.64
c root mean_squared_error: 5.91
c mean_absolute_error: 4.62


## process

In [6]:
df_AOD = pd.DataFrame(
{
    "SpatialScale":SpatialScale_ls,
    "TimeScale":TimeScale_ls,
    "Gas":Gas_ls,
    "R2":R2_ls,
    "RMSE":RMSE_ls,
    "MAE":MAE_ls,
    "AOD":AOD_ls
}
)
df_AOD["SpatialScale"] = df_AOD["SpatialScale"].str.upper()
df_AOD["TimeScale"] = df_AOD["TimeScale"].str.replace("monthly","Monthly")
df_AOD["TimeScale"] = df_AOD["TimeScale"].str.replace("Monthly_le","Monthly_D")

## merge and save as csv

In [7]:
pd.concat([df_ori,df_AOD]).to_csv("./data/LR_wo_aod.csv",index=False)